In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [7]:
df_origin = pd.read_csv('data/hood_page1to20_final_nodup.csv')
df = df_origin.copy()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35752 entries, 0 to 35751
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   35752 non-null  int64  
 1   user         35752 non-null  object 
 2   gender       33350 non-null  object 
 3   height       33350 non-null  object 
 4   weight       33350 non-null  object 
 5   item         35752 non-null  object 
 6   size         35752 non-null  object 
 7   star         35752 non-null  float64
 8   content      35752 non-null  object 
 9   size_eval    35183 non-null  object 
 10  bright_eval  35183 non-null  object 
 11  color_eval   35183 non-null  object 
 12  thick_eval   35183 non-null  object 
 13  cm           21905 non-null  object 
 14  총장           21905 non-null  float64
 15  어깨너비         20885 non-null  float64
 16  가슴단면         21905 non-null  float64
 17  소매길이         21798 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory usa

In [18]:
print('null값모두삭제:{}'.format(df.dropna(axis = 0).shape))
print('성별,키,몸무게 null 삭제:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()].shape))
print('성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())].shape))
print('성별,키,몸무게 null:{}'.format(df[df['gender'].isnull()&df['height'].isnull()&df['weight'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이4개 전부 null:{}'.format(df[df['총장'].isnull()&
   df['어깨너비'].isnull()&df['가슴단면'].isnull()&
   df['소매길이'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이 하나라도 null:{}'.format(df[df['총장'].isnull()|
   df['어깨너비'].isnull()|df['가슴단면'].isnull()|
   df['소매길이'].isnull()].shape))



null값모두삭제:(19511, 18)
성별,키,몸무게 null 삭제:(33350, 18)
성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:(20467, 18)
성별,키,몸무게 null:(2402, 18)
총장,어깨너비,가슴단면,소매길이4개 전부 null:(13847, 18)
총장,어깨너비,가슴단면,소매길이 하나라도 null:(14967, 18)


In [20]:
df_figure = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
df_figure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20467 entries, 100 to 35737
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   20467 non-null  int64  
 1   user         20467 non-null  object 
 2   gender       20467 non-null  object 
 3   height       20467 non-null  object 
 4   weight       20467 non-null  object 
 5   item         20467 non-null  object 
 6   size         20467 non-null  object 
 7   star         20467 non-null  float64
 8   content      20467 non-null  object 
 9   size_eval    20467 non-null  object 
 10  bright_eval  20467 non-null  object 
 11  color_eval   20467 non-null  object 
 12  thick_eval   20467 non-null  object 
 13  cm           20467 non-null  object 
 14  총장           20467 non-null  float64
 15  어깨너비         19611 non-null  float64
 16  가슴단면         20467 non-null  float64
 17  소매길이         20366 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory u

In [24]:
df_figure.iloc[: , 1:].to_csv('data/hoodtrain_sample.csv',index=False,encoding = "UTF-8")

In [61]:
df1 = pd.read_csv('data/hood_2_1_11page.csv')
df2 = pd.read_csv('data/hood_2_12_30page.csv')
df3 = pd.read_csv('data/hood_2_31_40page.csv')
df4 = pd.read_csv('data/hood_2_41_70page.csv')
df4.drop(['허리단면','허벅지단면','밑위','밑단단면'],axis = 1,inplace=True)
df5 = pd.read_csv('data/hood_2_71_90page.csv')
dfpage2 = pd.concat([df1,df2,df3,df4,df5])
dfpage2.to_csv('data/hood_2page.csv',encoding="UTF-8", index=False)



## 전처리에 들어갈 코드

* 사이즈 사전완성하고 돌릴 df완성

In [167]:
def crawlingdataconcat():
    df = pd.DataFrame()
    for i in range(1,21,1):
        try:
            df_before = pd.read_csv('data/hood_'+str(i)+'page.csv') #크롤링파일 불러오기
            print('df',str(i),': ',df_before.shape)
        except:
            df_before = pd.DataFrame() #아직크롤링되지 않은파일은 빈 df
            print('df',str(i),': ',df_before.shape)
            
        df = pd.concat([df,df_before]) #합치기
            

            
    print('df : ', df.shape)
    df.drop_duplicates(subset = None,keep = 'first', inplace = True,ignore_index = True)
    print('중복제거df : ', df.shape)#전체열이 같은 중복제거
    #성별, 키,몸무게 모두 null아니고 수치종류 적어도 하나 값이 있는 것들 filter
    df = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
    print('최종사용 data:{}'.format(df.shape))
    print('-'*10)
    return df

def crawlingdataprocessing(df):
    #kg,cm제거 및 수치타입으로 변경
    df["height"]=df["height"].replace('cm','',regex = True)
    df["weight"]=df["weight"].replace('kg','',regex = True)
    df= df.astype({'height':'float','weight':'float'})
    print('df전처리완료')
    print('-'*10)
    return df

def add_reviewcol(df):
    df = df.drop_duplicates(subset=['content'])#리뷰중복제거
    print('같은내용리뷰 중복제거 완료')
    df.reset_index(drop=True, inplace=True)

    df['review'] = str('')
    #외국어 리뷰 삭제
    i = 0; nohangul = []
    for i in range(df.shape[0]):
        text = re.sub('[^ㄱ-ㅣ가-힣]', '',df.iloc[i,8])
        if(text==''):
            nohangul.append(i)
    df = df.iloc[[True if i not in nohangul else False for i in range(df.shape[0])],:]
    df.reset_index(drop=True, inplace=True)

    i=0
    for i in range(df.shape[0]):
        text = df.iloc[i,7]
        text = re.sub(pattern='[^\w\s\n]', repl='', string=text) #특수문자 제거
        text = re.sub(pattern='[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z]', repl=' ', string=text) #숫자, 그이외 삭제
        text = re.sub(pattern='[ㄱ-ㅎㅏ-ㅣ]+', repl='', string=text) #단순 모음, 자음 삭제
        text = repeat_normalize(text, num_repeats=2) #불필요반복문자정규화
        #text = spacing(text) #띄어쓰기------------------------>써야함!
        df['review'][i] = text
    print('리뷰 전처리 열 추가')
    print('df shape:{}'.format(df.shape))
    print('-'*10)
    return df

df = crawlingdataconcat()
df =crawlingdataprocessing(df)
df = add_reviewcol(df)

df 1 :  (9000, 17)
df 2 :  (8808, 17)
df 3 :  (8370, 17)
df 4 :  (6690, 17)
df 5 :  (7649, 17)
df 6 :  (6419, 17)
df 7 :  (5687, 17)
df 8 :  (5715, 17)
df 9 :  (5102, 17)
df 10 :  (5154, 17)
df 11 :  (4318, 17)
df 12 :  (3671, 17)
df 13 :  (3389, 17)
df 14 :  (3673, 17)
df 15 :  (3103, 17)
df 16 :  (2906, 17)
df 17 :  (2658, 17)
df 18 :  (2445, 17)
df 19 :  (1903, 17)
df 20 :  (1734, 17)
df :  (98394, 17)
중복제거df :  (39203, 17)
최종사용 data:(22502, 17)
----------
df전처리완료
----------
같은내용리뷰 중복제거 완료
리뷰 전처리 열 추가
df shape:(20037, 18)
----------


In [ ]:
#최종 data = df

### 사이즈사전

In [77]:
##pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install py-hanspell
# !pip install PyKomoran
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from selenium import webdriver
import pandas as pd
from tqdm.notebook import tqdm
import time, urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
keys = Keys()
import numpy as np;import re, os, sys, json

from hanspell import spell_checker

from konlpy.tag import Kkma #형태소분석
from konlpy.tag import Okt #형태소분석
import soynlp; from soynlp.normalizer import * #정규화

from konlpy.tag import Komoran, Hannanum, Kkma, Okt
komoran = Komoran(); hannanum = Hannanum(); kkma = Kkma(); okt = Okt();
# from pykospacing import Spacing
# spacing = Spacing()



In [109]:
total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏이','핏','핏입니다',
      '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
      '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷','후드','후드티','핏은','좋아요핏','요핏도',
      '좋아요핏도','상의','요핏','폼','폭','사이즈','후드','옷','품','전체','품도','몸통','옷핏','상체','핏',
      '핏이구','핏이었','핏임','핏입',
      '핏입니','핏감','요핏','폼','후드핏','핏이','크기','상품','상의','핏더']
chongjang = ['길이','기장','총장','궁디','궁딩','엉덩이','밑','밑단','위아래','끝단','밑기장',
             '밑단이랑','기장이','기장입니','밑단','밑','기장감','요기장','총길이','총장','총기장',
            '길이감','끝','기장','길이']
shoulder = ['어깨핏','어께','어깨','너비','어깡','골격','바디','가로','넓이','등판','통',
            '어깨깡패','어깨라','어깨넓','어깨쪽','어께','몸집','어깨핏','어깨선','어깨라인','어깡',
            '어깨','통','몸통','골격','넓이','너비','등판']

chest = ['가슴','둘레','바디','가로','넓이','통','둘레','몸집','가슴팍','통','몸통']
arm = ['소매','팔도','손목','손','팔다리','팔목','팔','팔꿈치','팔이','요팔',
       '당팔','팔길이','팔다리','팔길','팔소매','손목','팔목','소매통','손','팔','소매','팔기장']

small = ['작습니다','짧았어요','짧습니다','짧으면','짧','작았어요','짧다고','쪼여서','작았으면',
      '작아도','작으면','작음','작아진','좁다고',
      '짧아요','짧게','짧다는','좁다는','작네','작기는','달라붙는','작은데','짧은데','짧음',
      '작긴','짧네요','작게','작아서','작은','짧고','짧아','좁은','타이트','짧았지만','붙어서','짧긴',
      '작은것','작아요','짧은','짧아서','짧지만','좁아','좁고','작네요','작아','쪼이는','좁아요','크롭느낌',
      '크롭하','작고','숏','쫄려','작다고','작다','좁아서','작','크롭','크롭된','미니','크롭해',
      '길었으면','컸으면','좁게','짧지' ,'작지','크롭이','핏되',
      '숏하','사이즈업하','크롭하','숏한','크롭하긴','크롭이','크롭해','크롭느낌','크롭된','사이즈업','짝',
       '작','숏','짧긴','크롭','크롭한','타이트','크롭이라','작긴',
       '작','짧','좁']

big = ['큰','긴','커요','크고','크긴','넉넉하고','길어서','넉넉하게','작다는'
     '넉넉한','클','길고','큰데','크지','크지만','부해','크니깐','크더라구요','컸어도','넓음','큼직하게',
     '박시하네요','넉넉할','헐렁하게','큼직하니','컸음','큽니','긴데','길어','길게','넓어서','넓어','박시하고','컸어요',
     '박시','크다고','크다는','넓게','넓고','커용','길다는','크지도','큼','크다','길었어요','넉넉함','큰데도','덮는데',
     '덮이고','컸고','벙벙해서','크더라고요','컸을','크거나','크더라','컸었는데','컸어용','컸습니다','덮네요',
     '큼지막해서','덥는','박시하게','크네요','클까','넉넉하구','박시하','컸는데','넉넉하네요','크게','크니까','넉넉한데','길어요',
     '컸지만','헐렁한','크네','넉넉해','펑퍼짐한','덮어용','박시하니','헐렁해','넉넉하지''넉넉하구요','커','큽니다','넉넉하니',
     '크','길지','박시한데','덮어요','길긴','넉넉해요','넓은','덮히는', '넓긴','덮어줘서','덮은','넉넉해서','큼직해서','기네','낙낙',
     '흘러내리는','덮어서','벙벙한','커서','큰것','박시핏','오퍼핏','낙낙해','여유핏','널널함','길','덮어','넉넉합니다','크면',
     '덮고','가오리','루즈핏','펑퍼짐','길었지만','헐렁하고','남아','아방핏','박스핏','널널','오바핏','덮습니다','넉넉히',
     '헐렁해서','접어도','접어','접어서','널','넓어요','덮고도','오버','와이드','낭낭','헐렁','접으면','짧았으면','접고',
     '버핏','접어야','잡아먹힌','수선해서','오버핏','오버사이즈','오버사이징해서','낙낙하다','낙낙한데','낙낙해요','오버핏이예요','오버핏입니당',
       '요박시하','컸어용','낙낙합니','어벙벙하','와이드핏','크니깐','크네요옷','널널해요','오버핏나와요','널럴하',
       '요넉넉하','엄청큼','여유핏','오버느낌','아방핏','오버핏은','넓긴','넉넉함','입니다오버핏','큽니다오버핏',
       '큽니다제','큽니당','오버해요','요박시한','헐렁하긴','헐렁거리','아방방한','아방방','박시합니','박시했다',
       '빅사이즈','받았습니다오버핏','빡시하','루즈핏으로','낭낭한','오버핏이긴','박스핏','널널함','어벙벙',
       '요넉넉한','오퍼핏','오버핏이구요','낙낙해','낭낭해서','넉넉하구','넉넉한데','아방해','오버핏이라서',
       '낭낭하','박시해','커용','넉넉해','펑퍼짐','박시하지','널널','좋아요오버핏','어벙벙한','널널하','오바핏',
       '오버하지','오버핏하','박시합니다','오버핏되','박시함','버핏','박시한','낙낙하니','박시해요','크네용',                     '낙낙해서','큼직해서','루즈해','느슨','널널합니다','아방아','아방아방','루즈핏이','박시하','세미오버',
       '세미오버핏','루즈해서','오버핏이라','루즈핏이라','오버핏이네요','오버핏입니','어벙','오버해','박시한데',
       '아방','오버핏이에요','예뻐요오버핏','큼지막','박시핏','오버해서','루즈한','오버핏인데','넉넉하구요','박시할',
       '큼직','오버한','오버핏이','낙낙하','아방하','롱','오버핏으로','낙낙한','오버핏입니다','박시해서','와이드',                     '요오버핏','넉넉해서','큽니','오버','오버하','루즈','루즈핏','박시','넉넉','길','크긴','아방한','널널한',
       '헐렁','길긴','와이드하','아방해서','세미오버사이즈',             
       '크','길','널','덮','넓','벙벙하','덮이','덮히','커다랗','덥히','넉넉해지','뒤집'             
       '넉넉']

In [116]:
# 두개의 사전이 합치고 나서 진행
# 중복제거
# 'n+2'개의 최종 리스트가 생성됨
def sizedict_nodup(oneOfRawList):
    print('중복제거 전:{}개'.format(len(oneOfRawList)))
    oneOfRawList = set(oneOfRawList)
    print('중복제거 후:{}개'.format(len(oneOfRawList)))
    print('-'*10)
    return list(oneOfRawList)

In [127]:
total = sizedict_nodup(total)
chongjang = sizedict_nodup(chongjang)
shoulder = sizedict_nodup(shoulder)
arm  = sizedict_nodup(arm)
chest = sizedict_nodup(chest)
small = sizedict_nodup(small)
big = sizedict_nodup(big)


중복제거 전:52개
중복제거 후:52개
----------
중복제거 전:20개
중복제거 후:20개
----------
중복제거 전:19개
중복제거 후:19개
----------
중복제거 전:16개
중복제거 후:16개
----------
중복제거 전:11개
중복제거 후:9개
----------
중복제거 전:76개
중복제거 후:76개
----------
중복제거 전:255개
중복제거 후:255개
----------


In [128]:
#최종 사이즈 사전 완성
def final_sizedict(size_figure_list):
    all_big = {}
    all_small = {}
    for i in size_figure_list:
        all_big[i] = big
    
    for i in size_figure_list:
        all_small[i] = small
    
    return all_big,all_small

In [129]:
total_big,total_small = final_sizedict(total)
chongjang_big,chongjang_small = final_sizedict(chongjang)
shoulder_big,shoulder_small = final_sizedict(shoulder)
arm_big,arm_small = final_sizedict(arm)
chest_big,chest_small = final_sizedict(chest)


In [133]:
chest_small

{'가슴팍': ['좁은',
  '사이즈업',
  '길었으면',
  '크롭하긴',
  '좁다고',
  '짧은',
  '숏',
  '짧고',
  '작긴',
  '짧으면',
  '크롭하',
  '크롭해',
  '크롭느낌',
  '작기는',
  '크롭이라',
  '크롭이',
  '짧아서',
  '숏한',
  '작으면',
  '짧습니다',
  '달라붙는',
  '작음',
  '작은',
  '짧아',
  '작지',
  '짧지',
  '짧긴',
  '크롭된',
  '작아',
  '작게',
  '작네',
  '좁다는',
  '짧',
  '짝',
  '작은것',
  '작아요',
  '짧아요',
  '미니',
  '짧은데',
  '작은데',
  '작았으면',
  '좁게',
  '짧지만',
  '좁',
  '짧았어요',
  '작아서',
  '좁아요',
  '좁고',
  '작아도',
  '작아진',
  '좁아',
  '짧았지만',
  '작다고',
  '쪼여서',
  '작',
  '짧네요',
  '타이트',
  '숏하',
  '사이즈업하',
  '좁아서',
  '짧다는',
  '작고',
  '컸으면',
  '짧음',
  '쪼이는',
  '붙어서',
  '짧게',
  '크롭한',
  '작습니다',
  '작았어요',
  '크롭',
  '쫄려',
  '작다',
  '짧다고',
  '핏되',
  '작네요'],
 '몸통': ['좁은',
  '사이즈업',
  '길었으면',
  '크롭하긴',
  '좁다고',
  '짧은',
  '숏',
  '짧고',
  '작긴',
  '짧으면',
  '크롭하',
  '크롭해',
  '크롭느낌',
  '작기는',
  '크롭이라',
  '크롭이',
  '짧아서',
  '숏한',
  '작으면',
  '짧습니다',
  '달라붙는',
  '작음',
  '작은',
  '짧아',
  '작지',
  '짧지',
  '짧긴',
  '크롭된',
  '작아',
  '작게',
  '작네',
  '좁다는',
  '짧',
  '짝',
  '작은것',
  '작아요',
  '짧아요',
  '미

### 사전으로 01추출해보기 여기부터 해보기 마구잡이

In [168]:
df_origin = df
df = df.head(10)

In [163]:
def get_front_keywords_back(keywords, review):
    
    
    df['front_keywords'] = str('')
    df['after_keywords'] = str('')
    for i in range(len(review)):
        for j in keywords:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                b = re.findall(j + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                df['front_keywords'][i] = str(a)
                df['after_keywords'][i] = str(b)
                #b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
            else:
                df['front_keywords'][i] = ''
                df['after_keywords'][i] = ''
                
                # if str(a)
                # for word in a:
                #     t = ()
                #     feature_temp.append(t + (word,keyword))
                # for ngram in b:
                #     t = ()
                #     feature_temp.append(t + (ngram,keyword))
                # for ngram in c:
                #     t = ()
                #     feature_temp.append(t + (ngram,keyword))

    return df

In [178]:
def get_front_keywords_back_test(keywords, review):
    
    
    df['front_keywords'] = str('')
    df['after_keywords'] = str('')
    for j in keywords:
        for i in range(len(review)):
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                print(a)
                print(b)
                print('-'*10)
            #     df['front_keywords'][i] = str(a)
            #     df['after_keywords'][i] = str(b)
            #     #c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
            # else:
            #     df['front_keywords'][i] = ''
            #     df['after_keywords'][i] = ''
                
             

    

In [179]:
df['review']

0    면이 톡톡해서 좋아요 너무 얇지도 두껍지도 않아요평상시 S입는데 오버핏으로 입고 싶...
1       자주사용할 회색후드티 찾고있었는데 가성비도 좋고 기본템으로 활용하기 좋아 구매했어요
2    요즘은 진짜 와릿이즌이 대세인가봐요직장동료가 요즘애들이 좋아하는 브랜드라고 해서 구...
3                      질 맞고 좋아요 색깔도 이쁘고 촌스럽지 않고 너무 좋아요
4    솔직히 받고나서는 건조기 돌릴거 감안해서 m살까 했는데 지금보니 s사길 잘한 것 같...
5    기모는 없는데 얇지도 두껍지도 않아요 옷은 좀 크게 나온 편인거 같아요 엉덩이 다 ...
6                        덩치 큰 초 남자아이인데 잘 맞아요 길이도 적당합니다
7              원단도 도톰하고 가을에서 겨울까지 입을수 있겠어요사이즈 넉넉하고 좋아요
8                           귀엽습니당 싸게 잘사고 편해서 기부니가 좋습니다
9          사이즈 적당하고 이뻐요기모가 아니라서 좀 춥긴하지만 패딩안에 입을거라 괜찮아요
Name: review, dtype: object

In [180]:
get_front_keywords_back_test(total, df['review'])

[]
[]
----------
[]
[]
----------
[]
[]
----------
['옷은 좀 크게']
['않아요 옷은']
----------
['핏으로 입고 싶다고해서딸내미']
[]
----------
['핏은 맘에 들어요']
['덮어요 핏은']
----------
['후드티 찾고있었는데 가성비도']
[]
----------
[]
[]
----------
[]
[]
----------


In [164]:
test = get_front_keywords_back(total, df['review'])

In [166]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20037 entries, 0 to 20036
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user            20037 non-null  object 
 1   gender          20037 non-null  object 
 2   height          20037 non-null  float64
 3   weight          20037 non-null  float64
 4   item            20037 non-null  object 
 5   size            20037 non-null  object 
 6   star            20037 non-null  float64
 7   content         20037 non-null  object 
 8   size_eval       20037 non-null  object 
 9   bright_eval     20037 non-null  object 
 10  color_eval      20037 non-null  object 
 11  thick_eval      20037 non-null  object 
 12  cm              20037 non-null  object 
 13  총장              20037 non-null  float64
 14  어깨너비            19214 non-null  float64
 15  가슴단면            20037 non-null  float64
 16  소매길이            20036 non-null  float64
 17  review          20037 non-null 

In [ ]:
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [160]:
len(test)

34309

In [ ]:
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review.encode('utf-8')):
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))

    return feature_temp


def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list


def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        # for negative in negative_word_emotion:
        #     if re.findall(negative,ngram):
        #         is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        # for negative in negative_word_emotion:
        #     if re.findall(negative,ngram):
        #         is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

## 비고

In [ ]:
def get_front_keywords_back(feature_keywords, review):
    
    feature_temp = []
    
    
    for i in range(len(review)):
        for keyword in feature_keywords:
            if re.findall(keyword, review[i]):
                a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])
                
                for ngram in a:
                    t = ()
                    feature_temp.append(t + (ngram,keyword))
                for ngram in b:
                    t = ()
                    feature_temp.append(t + (ngram,keyword))
                for ngram in c:
                    t = ()
                    feature_temp.append(t + (ngram,keyword))

    return feature_temp